## 1. Cài đặt thư viện (Setup)

In [1]:
# ------------------------------------------------------------------
# CELL 1: SETUP & INSTALLATION
# ------------------------------------------------------------------
import os
import sys
import site
import shutil

print("⚙️ Đang cài đặt nnU-Net và các thư viện phụ trợ...")
!pip install nnunetv2 hiddenlayer graphviz --quiet
print("✅ Cài đặt hoàn tất.")

⚙️ Đang cài đặt nnU-Net và các thư viện phụ trợ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 4.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Cell 2: Cấu hình Môi trường & Symlink Data (Configuration)

In [2]:
# ------------------------------------------------------------------
# CELL 2: CONFIGURATION & DATA PREPARATION
# ------------------------------------------------------------------
# 1. Cấu hình biến môi trường
os.environ['nnUNet_raw'] = "/kaggle/temp/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = "/kaggle/temp/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/kaggle/working/nnUNet_results" 
os.environ['nnUNet_compile'] = 'F' # FIX LỖI P100: Tắt torch.compile

# Tạo folder gốc
for path in [os.environ['nnUNet_raw'], os.environ['nnUNet_preprocessed'], os.environ['nnUNet_results']]:
    os.makedirs(path, exist_ok=True)

# 2. Xử lý Symlink cho Data (Kỹ thuật tiết kiệm bộ nhớ Kaggle)
print("🔗 Đang tạo Symlink cho dữ liệu...")

# --- Preprocessed Data ---
input_preprocessed_plans = "/kaggle/input/ds312-nnunet-edl/nnUNET_preprocessed/nnUNET_preprocessed/Dataset101_BraTS2020/nnUNetPlans_3d_fullres"
input_preprocessed_root = os.path.dirname(input_preprocessed_plans)
target_preprocessed_dataset = os.path.join(os.environ['nnUNet_preprocessed'], "Dataset101_BraTS2020")
os.makedirs(target_preprocessed_dataset, exist_ok=True)

# Link folder nặng
target_plans_dir = os.path.join(target_preprocessed_dataset, "nnUNetPlans_3d_fullres")
if not os.path.exists(target_plans_dir):
    os.symlink(input_preprocessed_plans, target_plans_dir)

# --- THÊM ĐOẠN NÀY VÀO CELL 2 ---
# 1.1 Link folder 'gt_segmentations' (Để chạy bước chấm điểm cuối cùng)
input_gt_dir = "/kaggle/input/ds312-nnunet-edl/nnUNET_preprocessed/nnUNET_preprocessed/Dataset101_BraTS2020/gt_segmentations"
target_gt_dir = os.path.join(target_preprocessed_dataset, "gt_segmentations")
if not os.path.exists(target_gt_dir):
    os.symlink(input_gt_dir, target_gt_dir)
    
# Copy file nhẹ (json/pkl)
for item in os.listdir(input_preprocessed_root):
    s = os.path.join(input_preprocessed_root, item)
    d = os.path.join(target_preprocessed_dataset, item)
    if os.path.isfile(s) and (item.endswith(".json") or item.endswith(".pkl")):
        shutil.copy(s, d)

# --- Raw Data (Optional nhưng tốt để verify) ---
input_raw_images = "/kaggle/input/ds312-nnunet-edl/nnUNET_raw/nnUNET_raw/Dataset101_BraTS2020/imagesTr"
input_raw_root = os.path.dirname(input_raw_images)
target_raw_dataset = os.path.join(os.environ['nnUNet_raw'], "Dataset101_BraTS2020")
os.makedirs(target_raw_dataset, exist_ok=True)

target_imagesTr = os.path.join(target_raw_dataset, "imagesTr")
if not os.path.exists(target_imagesTr):
    os.symlink(input_raw_images, target_imagesTr)

raw_json_path = os.path.join(input_raw_root, "dataset.json")
if os.path.exists(raw_json_path):
    shutil.copy(raw_json_path, os.path.join(target_raw_dataset, "dataset.json"))

print("✅ Cấu hình Data & Môi trường hoàn tất.")

🔗 Đang tạo Symlink cho dữ liệu...
✅ Cấu hình Data & Môi trường hoàn tất.


## Cell 4: Thực thi Training (Execution)

In [3]:
# ------------------------------------------------------------------
# CELL 4: START TRAINING
# ------------------------------------------------------------------
print("🚀 BẮT ĐẦU TRAINING (Fold 0 - 30 Epochs)...")
# Lưu ý: Cờ --npz sẽ lưu file .npz rất nặng. Nếu sợ tràn ổ cứng Kaggle thì xóa cờ này đi.
!nnUNetv2_train Dataset101_BraTS2020 3d_fullres 0 --npz

🚀 BẮT ĐẦU TRAINING (Fold 0 - 30 Epochs)...

############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-11-25 08:39:50.225366: do_dummy_2d_data_aug: False
2025-11-25 08:39:50.226885: Using splits from existing split file: /kaggle/temp/nnUNet_preprocessed/Dataset101_BraTS2020/splits_final.json
2025-11-25 08:39:50.227391: The split file contains 5 spli

## Cell 4b

In [ ]:
# # ------------------------------------------------------------------
# # CELL 4b: CHẤM ĐIỂM THỦ CÔNG (FIX LỖI LỆCH SỐ LƯỢNG FILE)
# # ------------------------------------------------------------------
# import os
# import shutil

# # 1. Định nghĩa đường dẫn
# gt_folder_full = "/kaggle/temp/nnUNet_preprocessed/Dataset101_BraTS2020/gt_segmentations"
# pred_folder = "/kaggle/working/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres/fold_0/validation"
# dataset_json = "/kaggle/temp/nnUNet_preprocessed/Dataset101_BraTS2020/dataset.json"
# plans_json = "/kaggle/temp/nnUNet_preprocessed/Dataset101_BraTS2020/nnUNetPlans.json"

# # Folder tạm để chứa GT khớp
# gt_folder_matched = "/kaggle/temp/matched_gt_validation"

# print("🔍 Đang chuẩn bị dữ liệu chấm điểm...")

# if os.path.exists(pred_folder) and os.path.exists(gt_folder_full):
#     # Bước 1: Tạo folder GT tạm
#     if os.path.exists(gt_folder_matched):
#         shutil.rmtree(gt_folder_matched)
#     os.makedirs(gt_folder_matched)
    
#     # Bước 2: Lọc danh sách file trong folder dự đoán
#     # Chỉ lấy file .nii.gz (bỏ qua json, pkl nếu có)
#     pred_files = [f for f in os.listdir(pred_folder) if f.endswith('.nii.gz')]
#     print(f"📂 Tìm thấy {len(pred_files)} file dự đoán (Validation Set).")
    
#     # Bước 3: Copy/Link các file GT tương ứng sang folder tạm
#     count = 0
#     for fname in pred_files:
#         src = os.path.join(gt_folder_full, fname)
#         dst = os.path.join(gt_folder_matched, fname)
        
#         if os.path.exists(src):
#             os.symlink(src, dst) # Tạo shortcut cho nhanh, ko cần copy
#             count += 1
#         else:
#             print(f"⚠️ Cảnh báo: Không tìm thấy GT cho file {fname}")
            
#     print(f"✅ Đã tạo folder GT khớp ({count} files) tại: {gt_folder_matched}")
    
#     # Bước 4: Chạy chấm điểm trên folder GT tạm này
#     print("🚀 Bắt đầu chấm điểm...")
#     !nnUNetv2_evaluate_folder "$gt_folder_matched" "$pred_folder" -djfile "$dataset_json" -pfile "$plans_json"
    
#     print("\n🎉 HOÀN TẤT! File summary.json đã được tạo trong folder validation.")
# else:
#     print("❌ Lỗi: Không tìm thấy folder dự đoán hoặc folder GT gốc.")

## Cell 5: Đóng gói & Xuất kết quả (Export)

In [4]:
# # ------------------------------------------------------------------
# # CELL 5: EXPORT RESULTS
# # ------------------------------------------------------------------
# print("📦 Đang đóng gói kết quả...")

# # 1. Nén Model Results
# output_folder = "/kaggle/working/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres"
# zip_name = "/kaggle/working/BraTS2020_EDL_Fold0_Results"

# if os.path.exists(output_folder):
#     shutil.make_archive(zip_name, 'zip', output_folder)
#     print(f"✅ Đã nén model: {zip_name}.zip")
# else:
#     print("⚠️ Không tìm thấy folder kết quả để nén (Có thể training bị lỗi).")

# # 2. Copy file splits_final.json
# src_split = "/kaggle/temp/nnUNet_preprocessed/Dataset101_BraTS2020/splits_final.json"
# dst_split = "/kaggle/working/splits_final.json"

# if os.path.exists(src_split):
#     shutil.copy(src_split, dst_split)
#     print(f"✅ Đã lưu splits_final.json tại: {dst_split}")
# else:
#     print("⚠️ Không tìm thấy file splits_final.json.")

# print("🎉 HOÀN TẤT! BẠN CÓ THỂ TẢI FILE VỀ TỪ TAB OUTPUT.")

📦 Đang đóng gói kết quả...
⚠️ Không tìm thấy folder kết quả để nén (Có thể training bị lỗi).
✅ Đã lưu splits_final.json tại: /kaggle/working/splits_final.json
🎉 HOÀN TẤT! BẠN CÓ THỂ TẢI FILE VỀ TỪ TAB OUTPUT.


In [5]:
# ------------------------------------------------------------------
# CELL 5: EXPORT RESULTS (CODE NÉN GIỮ NGUYÊN CẤU TRÚC FOLDER)
# ------------------------------------------------------------------
import shutil
import os

print("📦 Đang đóng gói kết quả...")

# 1. Nén Model (Giữ nguyên cấu trúc nnUNet_results/...)
# Chúng ta sẽ nén từ thư mục gốc /kaggle/working
output_zip_name = "/kaggle/working/BraTS2020_EDL_Fold0_Results"
root_dir = "/kaggle/working"
base_dir = "nnUNet_results" # Chỉ nén folder này và các con của nó

try:
    shutil.make_archive(output_zip_name, 'zip', root_dir, base_dir)
    print(f"✅ Đã nén thành công: {output_zip_name}.zip")
    print("   (Giải nén ra sẽ có sẵn folder nnUNet_results/Dataset101...)")
except Exception as e:
    print(f"❌ Lỗi khi nén: {e}")

# 2. Copy file splits_final.json (Vẫn giữ nguyên bước này để backup)
src_split = "/kaggle/temp/nnUNet_preprocessed/Dataset101_BraTS2020/splits_final.json"
dst_split = "/kaggle/working/splits_final.json"

if os.path.exists(src_split):
    shutil.copy(src_split, dst_split)
    print(f"✅ Đã lưu riêng splits_final.json tại: {dst_split}")
else:
    print("⚠️ Không tìm thấy file splits_final.json.")

print("🎉 HOÀN TẤT! BẠN CÓ THỂ TẢI FILE VỀ TỪ TAB OUTPUT.")

📦 Đang đóng gói kết quả...
✅ Đã nén thành công: /kaggle/working/BraTS2020_EDL_Fold0_Results.zip
   (Giải nén ra sẽ có sẵn folder nnUNet_results/Dataset101...)
✅ Đã lưu riêng splits_final.json tại: /kaggle/working/splits_final.json
🎉 HOÀN TẤT! BẠN CÓ THỂ TẢI FILE VỀ TỪ TAB OUTPUT.


## Cell 6: Hình ảnh so sánh MRI vs AI Prediction vs Uncertainty Map

In [ ]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
import json
import shutil

# --- QUAN TRỌNG: TẮT COMPILE CHO P100 ---
os.environ['nnUNet_compile'] = 'F'

# ==============================================================================
# ⚙️ KAGGLE CONFIGURATION (ĐÃ SỬA ĐƯỜNG DẪN)
# ==============================================================================
CONFIG = {
    # 1. Đường dẫn Model (Lấy từ folder working)
    "checkpoint_path": "/kaggle/working/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres/fold_0/checkpoint_best.pth",
    
    # 2. Đường dẫn Data (Lấy từ folder temp symlink)
    "image_folder": "/kaggle/temp/nnUNet_raw/Dataset101_BraTS2020/imagesTr",
    "label_folder": "/kaggle/temp/nnUNet_preprocessed/Dataset101_BraTS2020/gt_segmentations", # Dùng GT đã preprocessed cho tiện
    "split_file":   "/kaggle/temp/nnUNet_preprocessed/Dataset101_BraTS2020/splits_final.json",

    # 3. Đường dẫn lưu kết quả (Lưu vào working để tải về)
    "output_folder": "/kaggle/working/inference_results_batch",
    
    # 4. Cấu hình chạy
    "run_mode":         "validation_split",  # Chạy đúng tập Validation của Fold 0
    "test_range":       [0, 10], 
    "num_random":       5,        
    
    # 5. Tùy chọn hiển thị
    "save_images":      True,     
    "show_on_screen":   True,    
}
# ==============================================================================

def get_case_list(folder):
    if not os.path.exists(folder):
        raise FileNotFoundError(f"❌ Folder {folder} không tồn tại!")
    files = sorted([f for f in os.listdir(folder) if f.endswith("_0000.nii") or f.endswith("_0000.nii.gz")])
    case_ids = []
    for f in files:
        if f.endswith(".nii"): cid = f.replace("_0000.nii", "")
        else: cid = f.replace("_0000.nii.gz", "")
        case_ids.append(cid)
    return case_ids

def calculate_dice(pred_slice, gt_slice):
    p = (pred_slice > 0).astype(np.float32)
    g = (gt_slice > 0).astype(np.float32)
    intersection = np.sum(p * g)
    sum_areas = np.sum(p) + np.sum(g)
    if sum_areas == 0: return 1.0
    return (2.0 * intersection) / sum_areas

def visualize_comparison(case_id, mri_data, gt_data, pred_data, uncertainty, slice_idx=None):
    # Tìm slice có khối u lớn nhất để vẽ cho đẹp
    if slice_idx is None:
        sums_gt = np.sum(gt_data, axis=(0, 1, 2))
        sums_pred = np.sum(pred_data, axis=(0, 1))
        
        if sums_gt.max() > 0: slice_idx = np.argmax(sums_gt)
        elif sums_pred.max() > 0: slice_idx = np.argmax(sums_pred)
        else: slice_idx = gt_data.shape[3] // 2

    print(f"    📸 Drawing Slice: {slice_idx}")

    # Xoay ảnh cho đúng chiều (.T)
    img_slice = mri_data[0, :, :, slice_idx].T
    
    # GT data từ preprocessed folder thường có shape [C, X, Y, Z] hoặc [X, Y, Z]
    # Nếu gt_data là [1, X, Y, Z]
    if gt_data.ndim == 4:
        gt_slice = gt_data[0, :, :, slice_idx].T
    else:
        gt_slice = gt_data[:, :, slice_idx].T
        
    pred_slice = pred_data[:, :, slice_idx].T
    unc_slice = uncertainty[:, :, slice_idx].T

    dice = calculate_dice(pred_slice, gt_slice)
    ratio = (np.sum(pred_slice>0) / np.sum(gt_slice>0) * 100) if np.sum(gt_slice>0) > 0 else 0

    fig, ax = plt.subplots(1, 4, figsize=(24, 6))
    plt.suptitle(f"Case: {case_id} | Slice: {slice_idx}", fontsize=16, y=0.98)

    # 1. MRI
    ax[0].imshow(img_slice, cmap='gray', origin='lower')
    ax[0].set_title("MRI Input", fontsize=12)
    ax[0].axis('off')

    # 2. GT
    ax[1].imshow(img_slice, cmap='gray', origin='lower', alpha=0.6)
    if np.any(gt_slice): ax[1].imshow(gt_slice, cmap='Greens', origin='lower', alpha=0.6, interpolation='nearest')
    ax[1].set_title("Ground Truth", fontsize=12, color='green')
    ax[1].axis('off')

    # 3. Pred
    ax[2].imshow(img_slice, cmap='gray', origin='lower', alpha=0.6)
    if np.any(pred_slice): ax[2].imshow(pred_slice, cmap='jet', origin='lower', alpha=0.5, interpolation='nearest')
    ax[2].set_title(f"AI Prediction\nDice: {dice:.1%} | Area: {ratio:.0f}%", fontsize=12, color='blue')
    ax[2].axis('off')

    # 4. Uncertainty
    im = ax[3].imshow(unc_slice, cmap='hot', origin='lower', vmin=0, vmax=1.0)
    ax[3].set_title("Uncertainty Map", fontsize=12, color='red')
    ax[3].axis('off')
    plt.colorbar(im, ax=ax[3], fraction=0.046, pad=0.04)

    if CONFIG["save_images"]:
        os.makedirs(CONFIG["output_folder"], exist_ok=True)
        save_path = os.path.join(CONFIG["output_folder"], f"{case_id}_slice{slice_idx}.png")
        plt.savefig(save_path, bbox_inches='tight', dpi=100)
        print(f"    ✅ Saved: {save_path}")
    
    if CONFIG["show_on_screen"]:
        plt.show()
    plt.close()

def process_case(predictor, case_id):
    print(f"\n🔍 Processing: {case_id}...")
    
    # Tìm file ảnh input
    base_file = os.path.join(CONFIG["image_folder"], f"{case_id}_0000.nii")
    ext = ".nii" if os.path.exists(base_file) else ".nii.gz"
    
    image_files = [os.path.join(CONFIG["image_folder"], f"{case_id}_{i:04d}{ext}") for i in range(4)]
    
    # Tìm file nhãn (GT)
    gt_file = os.path.join(CONFIG["label_folder"], f"{case_id}{ext}")
    if not os.path.exists(gt_file):
        # Thử tìm trong preprocessed nếu raw không có
        gt_file = os.path.join(CONFIG["label_folder"], f"{case_id}.nii.gz")
    
    if not os.path.exists(gt_file):
        print(f"    ⚠️ Skipping {case_id}: Label file missing.")
        return

    # Preprocessing & Inference
    preprocessor = predictor.configuration_manager.preprocessor_class(verbose=False)
    # Load và crop dữ liệu
    data, seg, _ = preprocessor.run_case(image_files, gt_file, predictor.plans_manager, predictor.configuration_manager, predictor.dataset_json)
    
    # Predict
    data_tensor = torch.from_numpy(data).to(predictor.device)
    pred_logits = predictor.predict_logits_from_preprocessed_data(data_tensor)
    
    # Tính EDL Uncertainty
    evidence = F.softplus(pred_logits)
    alpha = evidence + 1
    S = torch.sum(alpha, dim=0)
    K = alpha.shape[0]
    uncertainty = (K / S).cpu().numpy()
    segmentation = torch.argmax(pred_logits, dim=0).cpu().numpy()
    
    visualize_comparison(case_id, data, seg, segmentation, uncertainty)

def get_validation_cases(fold=0):
    split_file = CONFIG["split_file"]
    if not os.path.exists(split_file):
        raise FileNotFoundError(f"❌ Không tìm thấy file split tại: {split_file}")
        
    with open(split_file, 'r') as f:
        splits = json.load(f)
    
    if fold >= len(splits):
        raise ValueError(f"❌ Fold {fold} không tồn tại!")
        
    val_keys = splits[fold]['val']
    print(f"📂 Đã load danh sách Validation Fold {fold}: {len(val_keys)} ca.")
    return val_keys

def main():
    print("🚀 --- BATCH INFERENCE STARTED ---")
    
    # Load Model
    predictor = nnUNetPredictor(
        tile_step_size=0.5, use_gaussian=True, use_mirroring=True,
        device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'),
        verbose=False
    )
    
    checkpoint_folder = os.path.dirname(os.path.dirname(CONFIG["checkpoint_path"]))
    # nnUNetPredictor tự tìm fold_X trong folder này
    predictor.initialize_from_trained_model_folder(checkpoint_folder, use_folds=(0,), checkpoint_name="checkpoint_best.pth")
    print(f"📂 Model Loaded.")

    if CONFIG["run_mode"] == "validation_split":
        cases_to_run = get_validation_cases(fold=0)
        # cases_to_run = cases_to_run[:5] # Bỏ comment nếu muốn test nhanh 5 ca đầu
        print(f"⚙️ Mode: VALIDATION SPLIT (Fold 0) -> Running {len(cases_to_run)} cases.")
    else:
        # Code cho mode random/range giữ nguyên
        pass

    for case_id in cases_to_run:
        try:
            process_case(predictor, case_id)
        except Exception as e:
            print(f"    ❌ Error processing {case_id}: {e}")

    print("\n✅ --- BATCH INFERENCE FINISHED ---")
    
    # Nén ảnh kết quả để tải về
    shutil.make_archive("/kaggle/working/inference_images", 'zip', CONFIG["output_folder"])
    print("📦 Đã nén ảnh kết quả: /kaggle/working/inference_images.zip")

if __name__ == "__main__":
    main()